In [1]:
from neo4j import GraphDatabase
from json import dumps
import time
from pandas import DataFrame

class KnowledgeRecommendation(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        self.session = self._driver.session()

    def close(self):
        self._driver.close()
        
#     @staticmethod
#     def _epoch_to_date(epoch):
#         try:
#             s, ms = divmod(int(epoch), 1000)  # (1236472051, 807)
#         except Exception as e:
#             raise e
        
#         return '{}.{:03d}'.format(time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(s)), ms)
    
    def serialize_movie(self,movie):
        return {
            'id': movie['id'],
            'title': movie['title'],
#             'summary': movie['description'],
            'released': movie.get('releaseDate'),
            'duration': movie['runtime'],
            'rated': movie['rated'],
            'imdbId': movie['imdbId'],
            'genre': movie['genre']
        }
    
    def movie(self,title):
        query = ('MATCH (movie:Movie) ' +
                 f'WHERE movie.title =~ "{title}" ' +
                 'RETURN movie')
        results = self.session.run(query)
        return [self.serialize_movie(record['movie']) for record in results.data()]
    
    def same_genre_movies(self, title):
        query = ('MATCH (movie:Movie) ' +
                 f'WHERE movie.title = "{title}" '+
                'WITH movie.genre as genre '+
                'MATCH (movies:Movie) '+
                'WHERE movies.genre = genre '+
                'RETURN movies.title, movies.releaseDate, movies.genre, movies.imdbId '+
                'LIMIT 100')
        results = self.session.run(query)
#         return [self.serialize_movie(record['movies']) for record in results.data()] # you should only return movies
        return results.data()
    
    def movies_by_movie_director(self, title):
        query = ('MATCH (movie:Movie)<-[:DIRECTED]-(director:Director) '+
                f'WHERE movie.title = "{title}" '+
                'MATCH (movies:Movie)<-[:DIRECTED]-(director) '+
                'RETURN movies.title, movies.releaseDate, movies.genre, movies.imdbId '+
                'LIMIT 100')
        results = self.session.run(query)
        return results.data()
    
    def movies_by_actor(self,actor):
        query = ('MATCH (movies:Movie)<-[:ACTS_IN]-(actor:Actor) '+
                f'WHERE actor.name = "{actor}" '+
                'RETURN movies.title, movies.releaseDate, movies.genre, movies.imdbId ')
        results = self.session.run(query)
        return results.data()
    
    def movies_by_actor_and_genre(self,actor, genre):
        query = ('MATCH (movies:Movie)<-[:ACTS_IN]-(actor:Actor) '+
                f'WHERE actor.name = "{actor}" and movies.genre="{genre}" '+
                'RETURN movies.title, movies.releaseDate, movies.genre, movies.imdbId ')
        results = self.session.run(query)
        return results.data()
    
    def movies_shared_actors_with_genre(self,actor, genre):
        query = ('MATCH (actor:Actor)-[:ACTS_IN]->(movie:Movie)<-[:ACTS_IN]-(actors:Actor) ' +
            f'WHERE actor.name = "{actor}" ' +
            'WITH actors ' +
            'MATCH (actors)-[:ACTS_IN]->(movies:Movie) ' +
           f'WHERE movies.genre="{genre}" ' +
            'RETURN movies.title, movies.releaseDate, movies.genre, movies.imdbId ' +
            'LIMIT 100 ' )
        print(query)
        results = self.session.run(query)
        return results.data()
        
    
graph = KnowledgeRecommendation("bolt://localhost:11004", "neo4j", "123456")

In [2]:
avatar = graph.movie("Jurassic Park")
print(avatar)

[{'id': '329', 'title': 'Jurassic Park', 'released': '739749600000', 'duration': 127, 'rated': None, 'imdbId': 'tt0107290', 'genre': 'Action'}]


In [ ]:
other_movies = graph.same_genre_movies("Jurassic Park")
DataFrame(other_movies)

In [ ]:
movies_director = graph.movies_by_movie_director("Jurassic Park")
DataFrame(movies_director)

In [ ]:
actor = graph.movies_by_actor("Michael J. Fox")
DataFrame(actor)

In [ ]:
actor_and_genre = graph.movies_by_actor_and_genre("Michael J. Fox", "Action")
DataFrame(actor_and_genre)

In [ ]:
movies_other_actors_genre = graph.movies_shared_actors_with_genre("Michael J. Fox", "Animation")
DataFrame(movies_other_actors_genre)

# Resources
## Tutorials
[Best tutorial ever](https://janakiev.com/blog/wikidata-mayors/)
[Navegate the statements](https://en.wikibooks.org/wiki/SPARQL/WIKIDATA_Qualifiers,_References_and_Ranks)


### Useful attributes for movies
Awardsrecieved https://www.wikidata.org/wiki/Property:P166
main subject https://www.wikidata.org/wiki/Property:P921
genre https://www.wikidata.org/wiki/Property:P136
box office https://www.wikidata.org/wiki/Property:P2142
        


In [3]:
op = 1
if op == 1:
    query = """SELECT ?movie ?movieLabel ?main_subjectLabel ?genreLabel ?box WHERE {
      ?movie wdt:P345 "tt0107290";
        wdt:P921 ?main_subject;
        wdt:P136 ?genre.
      ?movie p:P2142 ?statement.
      ?statement ps:P2142 ?box
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }"""
elif op==2:
    query = """
        SELECT 
          ?countryLabel ?population ?area ?medianIncome ?age
        WHERE {
          ?country wdt:P463 wd:Q458.
          OPTIONAL { ?country wdt:P1082 ?population }
          OPTIONAL { ?country wdt:P2046 ?area }
          OPTIONAL { ?country wdt:P3529 ?medianIncome }
          OPTIONAL { ?country wdt:P571 ?inception. 
            BIND(year(now()) - year(?inception) AS ?age)
          }
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }"""

In [4]:
import requests

url = 'https://query.wikidata.org/sparql'

r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

In [5]:
data

{'head': {'vars': ['movie',
   'movieLabel',
   'main_subjectLabel',
   'genreLabel',
   'box']},
 'results': {'bindings': [{'movie': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q167726'},
    'box': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
     'type': 'literal',
     'value': '1029528120'},
    'movieLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Jurassic Park'},
    'main_subjectLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'dinosaur'},
    'genreLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'action film'}},
   {'movie': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q167726'},
    'box': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
     'type': 'literal',
     'value': '1029528120'},
    'movieLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Jurassic Park'},
    'main_subjectLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'dinosaur'